## Notebook containing imputation concordance analysis of Finnish, CEU recombination maps

In [1]:
## Libraries

#import datashader as ds, pandas as pd
#import datashader.transfer_functions as tf
import rpy2
import numpy as np
import pandas as pd
import scipy as sp
#import datetime
#import holoviews as hv
#from holoviews import opts
#hv.extension('bokeh')
from PIL.Image import core as _imaging
import os, sys
from PIL import Image
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
#yaml.warnings({'YAMLLoadWarning': False})
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 5000)
pd.set_option('display.width', 1000)
# import scikit-allel
import allel
# check which version is installed
print(allel.__version__)

## Input Arguments

### Truth set

In [2]:
## Read the .raw file
truth  = pd.read_csv("/Users/shabbeerhassan/imput_concord/chr20_TRUTH_SET_masked_only.raw",
                         delimiter = ' ')

## Read the truth set's MAF data
truth_freq  = pd.read_csv("/Users/shabbeerhassan/imput_concord/chr20_TRUTH_SET_masked_only.frq",
                         delimiter = '\s+')

## Read the truth set's MAC data
truth_counts  = pd.read_csv("/Users/shabbeerhassan/imput_concord/chr20_TRUTH_SET_masked_only.frq.counts",
                         delimiter = '\s+')

## Reading variant list
SNP  = pd.read_csv("/Users/shabbeerhassan/imput_concord/chr20_TRUTH_SET_masked_only_variants.txt",
                         delimiter = ' ',
                         header=None,
                         names = ['chr', 'Positions'])
SNP = SNP.drop(SNP.columns[[0]], axis=1)


In [3]:
truth.head(5)
truth_freq.head(5)
truth_counts.head(5)
SNP.head(5)

FID      IID  PAT  MAT  SEX  PHENOTYPE  rs2002370_C  exm1517584_C  exm1517629_A  exm1517639_G  rs6078732_G  exm2254185_A  rs4239717_G  exm1517714_A  rs6045924_G  rs6035505_A  rs4815112_T  rs6051321_C  rs6051520_G  exm1517946_T  exm1517960_G  exm1517986_T  rs6115830_T  rs4815586_C  rs6051908_T  rs6116005_A  rs461944_C  rs188205_C  rs204686_A  rs6139752_G  rs6076864_C  rs6053783_A  rs6038382_G  rs6085580_T  rs6117457_C  rs6038642_G  exm1518366_A  exm1518411_C  rs6117615_G  rs594702_C  rs521448_G  rs7262185_T  rs4816023_A  rs11907018_A  rs12625348_C  rs6108123_C  rs6055833_G  rs6086545_A  rs500655_G  rs6056178_T  exm1518579_G  rs2317321_C  rs6039459_T  rs7261002_A  rs6077589_A  rs6057028_A  exm2272870_C  exm1518655_G  rs4813044_G  rs6078117_G  exm1518736_C  exm1518759_C  rs3818331_G  rs6134520_C  exm1518874_A  exm1518899_A  rs3764712_G  rs912113_T  rs4814223_C  rs8124053_C  rs3088080_T  rs1543423_C  exm1519013_T  rs2422579_G  rs2256229_G  rs11697395_A  exm1519088_C  rs11696739_A  \
0   FR02_2   FR02_2    0    0    0         -9            1             0             2           0.0          2.0             1          1.0             0          0.0          0.0          1.0            0          0.0             0             2           0.0          1.0            1            1            1           0         1.0         1.0          1.0            0            0            0            0            0            0           0.0             0          0.0           1         2.0            0          1.0             0           0.0            0          0.0            0         0.0            0             0          0.0            0            0            1          0.0           0.0             0          1.0          0.0             0             0          0.0            2           0.0             0            0           2            1            1            1          0.0             0          0.0          2.0             0           0.0             0   
1   FR02_6   FR02_6    0    0    0         -9            0             0             0           0.0          1.0             0          0.0             0          0.0          0.0          0.0            1          0.0             0             2           0.0          1.0            1            0            1           0         0.0         2.0          0.0            2            0            0            2            0            1           0.0             0          0.0           1         2.0            1          2.0             0           0.0            0          0.0            0         0.0            0             0          0.0            1            0            1          0.0           1.0             0          1.0          2.0             2             1          0.0            2           0.0             0            2           0            0            2            2          0.0             0          0.0          0.0             0           0.0             0   
2   FR02_8   FR02_8    0    0    0         -9            2             0             0           0.0          1.0             1          1.0             0          0.0          1.0          1.0            0          0.0             0             0           0.0          1.0            1            0            2           1         0.0         0.0          2.0            0            0            1            0            1            2           0.0             0          0.0           0         1.0            0          2.0             0           0.0            0          1.0            0         1.0            2             0          1.0            0            0            2          0.0           0.0             0          0.0          1.0             1             0          1.0            0           1.0             0            1           0            1            0            1          1.0             0          1.0          2.0             0           1.0             1   
3  

,CHR,SNP,A1,A2,MAF,NCHROBS
0,20,rs2002370,C,T,0.185900,21642
1,20,exm1517584,C,T,0.028090,21642
2,20,exm1517629,A,C,0.411500,21642
3,20,exm1517639,G,A,0.002496,21632
4,20,rs6078732,G,A,0.496100,21638


,CHR,SNP,A1,A2,C1,C2,G0
0,20,rs2002370,C,T,4023,17619,0
1,20,exm1517584,C,T,608,21034,0
2,20,exm1517629,A,C,8905,12737,0
3,20,exm1517639,G,A,54,21578,5
4,20,rs6078732,G,A,10734,10904,2


,Positions
0,91088
1,126149
2,139576
3,168558
4,186405


### Test Sets

In [ ]:
## NO REFPANEL when phasing with maps only - imputation with refpanels - maps differed each time
chr20_coreex_eagle_phased_NO_refpanel_beagle_imputed_SISUv2_500samples_HAPMAP_TEST_SET_ONLY.vcf
chr20_coreex_eagle_phased_NO_refpanel_beagle_imputed_SISUv2_500samples_FINNISH_map_TEST_SET_ONLY.vcf
chr20_coreex_eagle_phased_NO_refpanel_beagle_imputed_SISUv2_500samples_CONSTANT_map_TEST_SET_ONLY.vcf

chr20_coreex_eagle_phased_NO_refpanel_beagle_imputed_SISUv2_2700samples_HAPMAP_TEST_SET_ONLY.vcf
chr20_coreex_eagle_phased_NO_refpanel_beagle_imputed_SISUv2_2700samples_FINNISH_map_TEST_SET_ONLY.vcf
chr20_coreex_eagle_phased_NO_refpanel_beagle_imputed_SISUv2_2700samples_CONSTANT_map_TEST_SET_ONLY.vcf

chr20_coreex_eagle_phased_NO_refpanel_beagle_imputed_1KG_HAPMAP_TEST_SET_ONLY.vcf
chr20_coreex_eagle_phased_NO_refpanel_beagle_imputed_1KG_FINNISH_map_TEST_SET_ONLY.vcf
chr20_coreex_eagle_phased_NO_refpanel_beagle_imputed_1KG_CONSTANT_map_TEST_SET_ONLY.vcf

## REFPANEL for both phasing and imputation - only maps differed each time
chr20_coreex_eagle_phased_beagle_imputed_SISUv2_500samples_refpanel_HAPMAP_TEST_SET_ONLY.vcf
chr20_coreex_eagle_phased_beagle_imputed_SISUv2_500samples_refpanel_FINNISH_map_TEST_SET_ONLY.vcf
chr20_coreex_eagle_phased_beagle_imputed_SISUv2_500samples_refpanel_CONSTANT_map_TEST_SET_ONLY.vcf

chr20_coreex_eagle_phased_beagle_imputed_SISUv2_2700samples_refpanel_HAPMAP_TEST_SET_ONLY.vcf
chr20_coreex_eagle_phased_beagle_imputed_SISUv2_2700samples_refpanel_FINNISH_map_TEST_SET_ONLY.vcf
chr20_coreex_eagle_phased_beagle_imputed_SISUv2_2700samples_refpanel_CONSTANT_map_TEST_SET_ONLY.vcf

chr20_coreex_eagle_phased_beagle_imputed_1KG_refpanel_HAPMAP_TEST_SET_ONLY.vcf
chr20_coreex_eagle_phased_beagle_imputed_1KG_refpanel_FINNISH_map_TEST_SET_ONLY.vcf
chr20_coreex_eagle_phased_beagle_imputed_1KG_refpanel_CONSTANT_map_TEST_SET_ONLY.vcf

## Function for computing imputation concordance

In [ ]:
## ## Function for computing imputation concordance¶

## Logic:
## 1). Read Truth files: vcf, samples, freq, counts, ID
## 2). Read Test files: vcf, samples, freq, counts, ID
## 3). Function: imput_concord[Truth_vcf, Truth_samples, Truth_freq, Truth_counts, Truth_ID, 
##                            Test_vcf, Test_samples, Test_freq, Test_counts, Test_ID]
## 4). 


In [ ]:
### Read & Prepare TRUTH RAW file

# TRUTH dataset 
truth_temp <- truth[,c(7:2039)] # to check for the allele flipping later on
truth_temp2 <- truth[,c(-1:-6)]
truth_fid <- as.data.frame(truth[,1])
# Remove underscore, to match impute SNP names
colnames(truth_temp2) <- gsub(pattern="_.",replacement="",colnames(truth_temp2))
# Rename plink columns with BP from SNP file
for (i in 1:nrow(SNP)) {
  colnames(truth_temp2)[i] <- as.data.frame(SNP[i,])
}


# Subsetting same BP/SNP-id in TRUTH files as in IMPUTED files
test_pos <- test[,2]
truth_final <- truth_temp2[, colnames(truth_temp2) %in% test_pos] # subset truth dataset to same positions as in test imputed one
truth_final <- cbind(truth_fid, truth_final) #Get FID in truth
colnames(truth_final)[1] <- "ID" # Rename 1st column as the ID
ncol(truth_final)


# Subsetting same BP/SNP-id in SNP files as in IMPUTED files
SNP_final <- as.data.frame(SNP[SNP$BP %in% test_pos,])
colnames(SNP_final) <- "BP"


# Check if alleles are flipped between TRUTH & TEST SETS
flip_check <- as.data.frame(sapply(strsplit(colnames(truth_temp), split='_', fixed=TRUE), function(x) (x[1])))
flip_check[,2] <- as.data.frame(sapply(strsplit(colnames(truth_temp), split='_', fixed=TRUE), function(x) (x[2])))
flip_check[,3] <- as.data.frame(SNP)
colnames(flip_check) <- c("plink_id", "plink_snp", "plink_bp")
flip_check <- as.data.frame(flip_check[flip_check$plink_bp %in% test_pos,])
flip_check <- cbind(flip_check, test[,3:4])
colnames(flip_check) <- c("plink_id", "plink_snp", "plink_bp", "test_ref", "test_alt")
flipped <- flip_check[flip_check$plink_snp != flip_check$test_alt,]
nrow(flipped)
nrow(flip_check)


In [ ]:
### Read & Prepare IMPUTED VCF file

test_temp <- test[,c(-1:-4)]
ncol(test_temp)
nrow(test_temp)


# Create an data frame with positions in it
test_temp2 <- as.data.frame(matrix(test$V2, nrow = nrow(test_temp), ncol = 1)) 


# Splitting each column with GP into 3 others
for (i in 1:ncol(test_temp)) {
  temp <- test_temp[,i]
  # Use transform to convert from character to numeric
  # Check using sapply(test_temp2, mode)
  test_temp2 <- as.data.frame(cbind(test_temp2, transform(do.call("rbind", strsplit(temp, ",")), stringsAsFactors=F))) 
}
ncol(test_temp2)


# Converting an entire dataframe from factor to numeric, while preserving the numeric decimal values
test_temp2 <- as.data.frame(sapply(test_temp2, function(x) as.numeric(as.character(x))))


# Get the dosages from GP
test_temp3 <- test[,c(2:4)]
ncol(test_temp2)
test_temp3 <- cbind(test_temp3,
                    sapply(seq(2,32464,3),
                             function(x)
                               test_temp2[,x]*0+
                               test_temp2[,x+1]*1+
                               test_temp2[,x+2]*2))
colnames(test_temp3) <- c("BP","Ref","Alt",truth_sample_id$V1)


# Transpose into a dataframe with BP in rows & ID in columns
test_final <- as.data.frame(t(test_temp3[,-1:-3]))
colnames(test_final) <- test_temp3$BP
rownames(test_final) <- seq(1, nrow(test_final), 1)
test_final$ID <- test_sample_id$V1
ncol(test_final)
nrow(test_final)

In [ ]:
# Concordance Calculation -------------------------------------------------------------

correl <- as.data.frame(matrix(NA, nrow = nrow(SNP_final), ncol = 1))
cor <- as.numeric(x=NA)

# Calculate correlation between Imputed and true dosages
for (i in 1:nrow(SNP_final)) {
  # Match by ID and subset SNP
  d <- merge(test_final[ ,c("ID",SNP_final[i,])], truth_final[ ,c("ID",SNP_final[i,])], by="ID")
  colnames(d) <- c("ID","Imputed","Typed")
  cnt_matching_samples <- nrow(d)
  
  #Keep only complete.cases
  d <- d[complete.cases(d),]
  cnt_matching_samples_exclNA <- nrow(d)
  
  #Correlation
  correl[i,1] <- cor(d$Typed, d$Imputed) 
}
correl_mat <- cbind(SNP_final, correl)
colnames(correl_mat) <- c("POS", "Correlation")


# Take absolute value of the flipped alleles to get rid of negative correlation
for (i in rownames(flipped)) {
  correl_mat[i,]$Correlation <- abs(correl_mat[i,]$Correlation)
}
correl_mat <- na.omit(correl_mat)

### Read MAF 
freq_final <- freq[, c(-1,-3,-4,-6)]
freq_final <- cbind(correl_mat, freq_final)
truth_freq_final <- cbind(truth_freq, SNP)
truth_freq_final <- as.data.frame(truth_freq_final[truth_freq_final$BP %in% test_pos,])


### Read MAC
counts_temp <- counts
#for (i in rownames(flipped)) {
#  counts_temp[i,]$C1 = counts_temp[i,]$C2
#}
counts_temp <- counts_temp[complete.cases(counts_temp),]
truth_counts_final <- cbind(truth_counts, SNP)
truth_counts_final <- as.data.frame(truth_counts_final[truth_counts_final$BP %in% test_pos,])

In [ ]:
## Prepare FINAL DATASET
final_temp <- as.data.frame(cbind(counts_temp$CHR, test$V2, 
                             counts_temp$C1, truth_counts_final$C1, freq_final$MAF,
                             truth_freq_final$MAF, freq$NCHROBS, correl_mat$Correlation))
final_temp <- as.data.frame(sapply(final_temp, function(x) as.numeric(as.character(x))))

# Categorize different MAF's
attach(final_temp)
final_temp$V9[V5 <=  0.005] <- "<0.5%"
final_temp$V9[V5 >  0.005 & V5 <=  0.01] <- "0.5%-1%"
final_temp$V9[V5 >  0.01 & V5 <=  0.025] <- "1%-2.5%"
final_temp$V9[V5 >  0.025 & V5 <=  0.05] <- "2.5%-5%"
final_temp$V9[V5 > 0.05] <- ">5%"

#final_temp$V9[V5 <=  0.01] <- "Rare-Freq"
#final_temp$V9[V5 >  0.01 & V5 <=  0.05] <- "Low-Freq"
#final_temp$V9[V5 > 0.05] <- "Common-Freq"
detach(final_temp)
final <- as.data.frame(final_temp)
colnames(final) <- c("CHR", "POS", "Test_Counts", "Truth_Counts",
                     "Test_MAF", "Truth_MAF", "Total", "Correlation", "Type")

# Values for individual files
ne_1k <- final
ne_1k$Correlation <- abs(ne_1k$Correlation)
write.table(ne_1k, "~/Dropbox/ne_1k.txt", sep="\t", row.names=FALSE)
colnames(ne_1k) <- c("CHR", "POS", "Test_Counts", "Truth_Counts",
                     "Test_MAF", "Truth_MAF", "Total", "Correlation", "Type")
ne_2k <- final
ne_2k$Correlation <- abs(ne_2k$Correlation)
write.table(ne_2k, "~/Dropbox/ne_2k.txt", sep="\t", row.names=FALSE)
colnames(ne_2k) <- c("CHR", "POS", "Test_Counts", "Truth_Counts",
                     "Test_MAF", "Truth_MAF", "Total", "Correlation", "Type")
ne_3k <- final
ne_3k$Correlation <- abs(ne_3k$Correlation)
write.table(ne_3k, "~/Dropbox/ne_3k.txt", sep="\t", row.names=FALSE)
colnames(ne_3k) <- c("CHR", "POS", "Test_Counts", "Truth_Counts",
                     "Test_MAF", "Truth_MAF", "Total", "Correlation", "Type")

ne_4k <- final
ne_4k$Correlation <- abs(ne_4k$Correlation)
write.table(ne_4k, "~/Dropbox/ne_4k.txt", sep="\t", row.names=FALSE)
colnames(ne_4k) <- c("CHR", "POS", "Test_Counts", "Truth_Counts",
                     "Test_MAF", "Truth_MAF", "Total", "Correlation", "Type")

ne_5k <- final
ne_5k$Correlation <- abs(ne_5k$Correlation)
write.table(ne_5k, "~/Dropbox/ne_5k.txt", sep="\t", row.names=FALSE)
colnames(ne_5k) <- c("CHR", "POS", "Test_Counts", "Truth_Counts",
                     "Test_MAF", "Truth_MAF", "Total", "Correlation", "Type")

ne_10k <- final
ne_10k$Correlation <- abs(ne_10k$Correlation)
write.table(ne_10k, "~/Dropbox/ne_10k.txt", sep="\t", row.names=FALSE)
colnames(ne_10k) <- c("CHR", "POS", "Test_Counts", "Truth_Counts",
                     "Test_MAF", "Truth_MAF", "Total", "Correlation", "Type")

ne_15k <- final
ne_15k$Correlation <- abs(ne_15k$Correlation)
write.table(ne_15k, "~/Dropbox/ne_15k.txt", sep="\t", row.names=FALSE)
colnames(ne_15k) <- c("CHR", "POS", "Test_Counts", "Truth_Counts",
                     "Test_MAF", "Truth_MAF", "Total", "Correlation", "Type")

ne_20k <- final
ne_20k$Correlation <- abs(ne_20k$Correlation)
write.table(ne_20k, "~/Dropbox/ne_20k.txt", sep="\t", row.names=FALSE)
colnames(ne_20k) <- c("CHR", "POS", "Test_Counts", "Truth_Counts",
                     "Test_MAF", "Truth_MAF", "Total", "Correlation", "Type")

ne_25k <- final
ne_25k$Correlation <- abs(ne_25k$Correlation)
write.table(ne_25k, "~/Dropbox/ne_25k.txt", sep="\t", row.names=FALSE)
colnames(ne_25k) <- c("CHR", "POS", "Test_Counts", "Truth_Counts",
                     "Test_MAF", "Truth_MAF", "Total", "Correlation", "Type")

ne_30k <- final
ne_30k$Correlation <- abs(ne_30k$Correlation)
write.table(ne_30k, "~/Dropbox/ne_30k.txt", sep="\t", row.names=FALSE)
colnames(ne_30k) <- c("CHR", "POS", "Test_Counts", "Truth_Counts",
                     "Test_MAF", "Truth_MAF", "Total", "Correlation", "Type")

ne_100k <- final
ne_100k$Correlation <- abs(ne_100k$Correlation)
write.table(ne_100k, "~/Dropbox/ne_100k.txt", sep="\t", row.names=FALSE)
colnames(ne_100k) <- c("CHR", "POS", "Test_Counts", "Truth_Counts",
                     "Test_MAF", "Truth_MAF", "Total", "Correlation", "Type")

ne_200k <- final
ne_200k$Correlation <- abs(ne_200k$Correlation)
write.table(ne_200k, "~/Dropbox/ne_200k.txt", sep="\t", row.names=FALSE)
colnames(ne_200k) <- c("CHR", "POS", "Test_Counts", "Truth_Counts",
                     "Test_MAF", "Truth_MAF", "Total", "Correlation", "Type")

ne_400k <- final
ne_400k$Correlation <- abs(ne_400k$Correlation)
write.table(ne_400k, "~/Dropbox/ne_400k.txt", sep="\t", row.names=FALSE)
colnames(ne_400k) <- c("CHR", "POS", "Test_Counts", "Truth_Counts",
                     "Test_MAF", "Truth_MAF", "Total", "Correlation", "Type")

ne_600k <- final
ne_600k$Correlation <- abs(ne_600k$Correlation)
write.table(ne_600k, "~/Dropbox/ne_600k.txt", sep="\t", row.names=FALSE)
colnames(ne_600k) <- c("CHR", "POS", "Test_Counts", "Truth_Counts",
                     "Test_MAF", "Truth_MAF", "Total", "Correlation", "Type")

ne_800k <- final
ne_800k$Correlation <- abs(ne_800k$Correlation)
write.table(ne_800k, "~/Dropbox/ne_800k.txt", sep="\t", row.names=FALSE)
colnames(ne_800k) <- c("CHR", "POS", "Test_Counts", "Truth_Counts",
                     "Test_MAF", "Truth_MAF", "Total", "Correlation", "Type")

ne_1m <- final
ne_1m$Correlation <- abs(ne_1m$Correlation)
write.table(ne_1m, "~/Dropbox/ne_1m.txt", sep="\t", row.names=FALSE)
colnames(ne_1m) <- c("CHR", "POS", "Test_Counts", "Truth_Counts",
                     "Test_MAF", "Truth_MAF", "Total", "Correlation", "Type")